# 06-03: Slack Bot 开发 (Bolt 框架)使用 @slack/bolt 框架创建 Slack Bot。

In [ ]:
// 安装: npm install @slack/bolt
import { App } from '@slack/bolt';
// ========== 1. 基础 Bot 创建 ==========
const app = new App({
  token: process.env.SLACK_BOT_TOKEN,
  signingSecret: process.env.SLACK_SIGNING_SECRET,
  socketMode: true,
  appToken: process.env.SLACK_APP_TOKEN,
});
// 启动应用
// (async () => {
//   await app.start(process.env.PORT || 3000);
//   console.log('⚡️ Bolt app is running!');
// })();

In [ ]:
// ========== 2. 消息监听 ==========
// 监听所有消息
app.message(async ({ message, say }) => {
  console.log('Received message:', message);
  // say() sends a message to the same channel
});
// 监听特定消息
app.message('hello', async ({ message, say }) => {
  await say(`Hey there <@${message.user}>!`);
});
// 使用正则表达式
app.message(/^(hi|hello|hey).*/i, async ({ context, say }) => {
  const greeting = context.matches[0];
  await say(`${greeting}, how are you?`);
});

In [ ]:
// ========== 3. Slash Commands ==========
app.command('/echo', async ({ command, ack, say }) => {
  // Acknowledge command request
  await ack();
  
  await say(`You said: ${command.text}`);
});
// 带参数的 command
app.command('/weather', async ({ command, ack, respond }) => {
  await ack();
  
  const city = command.text || 'Beijing';
  
  // 调用天气 API
  // const weather = await fetchWeather(city);
  
  await respond({
    text: `Weather in ${city}: Sunny, 25°C`,
    response_type: 'in_channel' // 让所有人看到
  });
});

In [ ]:
// ========== 4. Block Kit UI ==========
app.command('/poll', async ({ command, ack, say }) => {
  await ack();
  
  await say({
    blocks: [
      {
        type: 'section',
        text: {
          type: 'mrkdwn',
          text: '*What is your favorite programming language?*'
        }
      },
      {
        type: 'actions',
        elements: [
          {
            type: 'button',
            text: {
              type: 'plain_text',
              text: 'TypeScript'
            },
            value: 'typescript',
            action_id: 'vote_typescript'
          },
          {
            type: 'button',
            text: {
              type: 'plain_text',
              text: 'Python'
            },
            value: 'python',
            action_id: 'vote_python'
          },
          {
            type: 'button',
            text: {
              type: 'plain_text',
              text: 'Rust'
            },
            value: 'rust',
            action_id: 'vote_rust'
          }
        ]
      }
    ]
  });
});

In [ ]:
// ========== 5. 按钮交互处理 ==========
app.action('vote_typescript', async ({ body, ack, say }) => {
  await ack();
  const user = body.user.name;
  await say(`${user} voted for TypeScript!`);
});
app.action('vote_python', async ({ body, ack, say }) => {
  await ack();
  await say(`${body.user.name} voted for Python!`);
});
app.action('vote_rust', async ({ body, ack, say }) => {
  await ack();
  await say(`${body.user.name} voted for Rust!`);
});
// 处理所有按钮点击
app.action(/vote_.*/, async ({ ack }) => {
  await ack();
});

In [ ]:
// ========== 6. 事件监听 ==========
// 成员加入
app.event('team_join', async ({ event, client }) => {
  try {
    const result = await client.chat.postMessage({
      channel: event.user.id,
      text: `Welcome to the team, <@${event.user.id}>! 🎉`
    });
    console.log(result);
  } catch (error) {
    console.error(error);
  }
});
// 反应添加
app.event('reaction_added', async ({ event }) => {
  console.log(`Reaction ${event.reaction} added by ${event.user}`);
});
// ========== 7. 快捷菜单 ==========
app.shortcut('open_modal', async ({ shortcut, ack, client }) => {
  await ack();
  
  try {
    await client.views.open({
      trigger_id: shortcut.trigger_id,
      view: {
        type: 'modal',
        callback_id: 'submit_modal',
        title: {
          type: 'plain_text',
          text: 'My App'
        },
        blocks: [
          {
            type: 'input',
            block_id: 'input_block',
            element: {
              type: 'plain_text_input',
              action_id: 'input_action'
            },
            label: {
              type: 'plain_text',
              text: 'Enter something'
            }
          }
        ],
        submit: {
          type: 'plain_text',
          text: 'Submit'
        }
      }
    });
  } catch (error) {
    console.error(error);
  }
});

In [ ]:
// ========== 8. Modal 提交处理 ==========
app.view('submit_modal', async ({ ack, body, view }) => {
  await ack();
  
  const userInput = view.state.values.input_block.input_action.value;
  const user = body.user.id;
  
  // 处理提交的数据
  console.log(`User ${user} submitted: ${userInput}`);
  
  // 可以发送私信
  // await client.chat.postMessage({
  //   channel: user,
  //   text: `You submitted: ${userInput}`
  // });
});